In [708]:
import ta
import trendet
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, timezone
import sys
sys.path.append('/Users/jp/Desktop/Investment/utils')
import utils as utils
import math
from string import ascii_uppercase
from itertools import product
import os
from binance.client import Client
from binance.exceptions import *
import requests as requests
import time as time
import matplotlib.dates as mdates

In [709]:
class Macd_long_backtester_sma_sltp():
    '''
    Macd class for backtesting strategies
    How to use this class:
    1) Use 'prepare_data' method:
        INPUTS
        - start
        - end
        - interval
        
        COLUMNS CREATED
        - log_returns_hold
        - multiple_hold_acum
        - position
        - inv_sign
    '''
    
    def __init__(self, symbol=None, interval=None):
        
        """Macd long backtester constructor
        :param symbol: symbol from Binance from which to extract the data, .i.e. 'BTCUSDT'
        :type symbol: str.
        :param data: a dataframe with all the data extract from the Binance API for the selected function inputs.
        :type data: DataFrame
        :param data_uptrend: a dataframe with data extracted from self.data but only for the type of trend
        :type data_uptrend: str.
        :param data_downtrend: a dataframe with data extracted from self.data but only for the type of trend
        :type data_down_trend: str.
        :param data_sideways: a dataframe with data extracted from self.data but only for the type of trend
        :type data_sideways: str.
        """
        print('class version 0.0 is being used')
        self.symbol = symbol
        self.data_init = pd.DataFrame()
        self.start = None
        self.end = None
        self.interval = interval
        self.from_time = None
        self.to_time = None
        self.ema_slow = None
        self.ema_fast = None
        self.ema_signal = None
        self.sma_slow = None
        self.sma_fast = None
        self.opt_results = None
        self.opt_ini_time_str = None
        self.opt_minutes_elapsed = None
        self.opt_max = None
        self.opt_max_net = None
        self.sltp_dates = []
        
    def __repr__(self):
        return f"Macd_long_backtester(symbol={self.symbol})"

    def prepare_data(self, start=None, end=None):
        '''
        REMARK: Introduced time must be in Tokyo time (UTC+9) but the calculations will be in UTC
        Prepare all the fields of data necessary for the study. The interval of dates to be studied is the one
        given when delclaring the class. To prepare another interval of dates, please create another class instance.
        :param start: a string with the following format ""%Y-%m-%d-%H:%M" .i.e. "2022-01-29-20:00"
        :type start: str.
        :param end: a string with the following format ""%Y-%m-%d-%H:%M" .i.e. "2022-02-29-20:00"
        :type end: str.
        :param interval: string among the followings: ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
        :type interval: str.
        '''
        
        self.start = start
        self.end = end
        from_time = int(datetime.strptime(start, "%Y-%m-%d-%H:%M").timestamp()*1000)
        to_time = int(datetime.strptime(end, "%Y-%m-%d-%H:%M").timestamp()*1000)
        self.from_time = from_time
        self.to_time = to_time
        self.data_init = utils.get_history_v2(symbol=self.symbol, interval=self.interval, start=from_time, end=to_time)[0]
        self.data_init['log_returns_hold'] = np.log(self.data_init.Close.div(self.data_init.Close.shift(1)))
        self.data_init['multiple_hold_acum'] = np.exp(self.data_init.log_returns_hold.cumsum())
        #initialize positions and sign_inv
        self.data_init['position'] = 0
        self.data_init['inv_sign'] = 0
        
    def execute_backtest(self, start=None, ema_slow=None, ema_fast=None, ema_signal=None, sma_slow=None, sma_fast=None, sl=None, tp=None):
        '''
        REQUIREMENT: execute after "prepare_data" method
        :param start: a string with the following format ""%Y-%m-%d-%H:%M" .i.e. "2022-01-29-20:00"
        :type start: str.
        :param end: a string with the following format ""%Y-%m-%d-%H:%M" .i.e. "2022-02-29-20:00"
        :type end: str.
        '''
        #Prepare pre-data to start with non NaN at the backtesting period
        from_time_obj = datetime.strptime(start, "%Y-%m-%d-%H:%M")
#       ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
        extra_periods = 0
        ema_diff = ema_slow + ema_signal + 2
        
        # Necessary extra periods to have enough data to calculate the strategy parameters at the present moment
        if (ema_diff >= sma_slow):
            extra_periods = ema_diff
        else:
            extra_periods = sma_slow
        
        td = timedelta()
        if 'm' in self.interval:
            num_min = int(self.interval.replace('m',''))
            td = timedelta(minutes=num_min*(extra_periods))
        if 'h' in self.interval:
            num_h = int(self.interval.replace('h',''))
            td = timedelta(hours=num_h*(extra_periods))
        if 'd' in self.interval:
            num_day = int(self.interval.replace('d',''))
            td = timedelta(days=num_day*(extra_periods))
        if 'w' in self.interval:
            num_week = int(self.interval.replace('w',''))
            td = timedelta(weeks=num_week*(extra_periods))
        if 'M' in self.interval:
            num_week_m = int(self.interval.replace('M',''))
            td = timedelta(weeks=num_week_m * 4 * (extra_periods))
        pre_from_time_obj = from_time_obj - td
        pre_from_time = int((from_time_obj - td).timestamp()*1000)
        self.data_init_pre = utils.get_history_v2(symbol=self.symbol, interval=self.interval, start=pre_from_time, end=self.to_time)[0]
        #Assign the MACD parameters to the class to see which parameters have been used last
        self.ema_slow = ema_slow
        self.ema_fast = ema_fast
        self.ema_signal = ema_signal
        #obtaining MACD instance from python ta
        macd_diff = ta.trend.MACD(close=self.data_init_pre.Close, window_slow=ema_slow, window_fast=ema_fast, window_sign=ema_signal, fillna=False).macd_diff()
        macd_macd = ta.trend.MACD(close=self.data_init_pre.Close, window_slow=ema_slow, window_fast=ema_fast, window_sign=ema_signal, fillna=False).macd()
        macd_signal = ta.trend.MACD(close=self.data_init_pre.Close, window_slow=ema_slow, window_fast=ema_fast, window_sign=ema_signal, fillna=False).macd_signal()
        #assigning the values of macd to ticker dataframe
        self.data_init_pre['macd_diff'] = macd_diff
        self.data_init_pre['macd_macd'] = macd_macd
        self.data_init_pre['macd_signal'] = macd_signal         
        #assign SMAs values to class attributes
        self.sma_fast = sma_fast
        self.sma_slow = sma_slow       
        #calculating SMA fast and slow
        sma_fast_ps = ta.trend.SMAIndicator(close=self.data_init_pre.Close, window=self.sma_fast, fillna=False).sma_indicator()
        sma_slow_ps = ta.trend.SMAIndicator(close=self.data_init_pre.Close, window=self.sma_slow, fillna=False).sma_indicator()
        #assigning the smas values to ticker dataframe
        self.data_init_pre['sma_fast_ps'] = sma_fast_ps
        self.data_init_pre['sma_slow_ps'] = sma_slow_ps
        #copy the necessary data from the present moment
        self.data_init = self.data_init_pre.loc[from_time_obj:].copy()
        #Initialize again buy-and-hold parameters for new data
        self.data_init['log_returns_hold'] = np.log(self.data_init.Close.div(self.data_init.Close.shift(1)))
        self.data_init['multiple_hold_acum'] = np.exp(self.data_init.log_returns_hold.cumsum())
        #initialize again positions and sign_inv for new data
        self.data_init['position'] = 0
        self.data_init['inv_sign'] = 0
        self.data_init['sma_conf'] = 0
        self.data_init['sl_flag'] = None
        self.data_init['tp_flag'] = None
        
        # a sell signal appears when there is a macd_cross downwards and it is confirmed by: sma_fast_ps > sma_slow_ps
        #stablish neutral conditions
        
        #macd cross neutral
        ht_pos = self.data_init.macd_diff.shift(1) > 0
        ht_plusone_neg = self.data_init.macd_diff < 0        
       
        #sma neutral confirmation
        sma_conf_neutral = self.data_init.sma_fast_ps < self.data_init.sma_slow_ps
        
        #stablish neutral positions
        self.data_init.loc[ht_pos & ht_plusone_neg, 'inv_sign'] = -1
        self.data_init.loc[sma_conf_neutral, 'sma_conf'] = -1
        
        #macd cross long
        ht_neg = self.data_init.macd_diff.shift(1) < 0
        ht_plusone_pos = self.data_init.macd_diff > 0
        
        #sma long confirmation
        sma_conf_long = self.data_init.sma_fast_ps > self.data_init.sma_slow_ps        
        
        #stablish buy positions
        self.data_init.loc[ht_neg & ht_plusone_pos, 'inv_sign'] = 1
        self.data_init.loc[sma_conf_long, 'sma_conf'] = 1
        
        sltp_flag = False
        #Loop to stablish a long position when a signal appears and all the way till a neutral signal is detected
        for index, data in self.data_init.iterrows():
            # 3 conditions from left to right:
            # macd upwards cross
            # confirmation of sma
            # stop loss or take profit it is not considered in first iteration, just in sub loop
            if (data.inv_sign == 1 and data.sma_conf == 1):
                # stablish long position at 'index' position
                self.data_init.loc[index, 'position'] = 1
                # loop to stablish all dates that we will go long since a neutral signal is detected, then break
                # and go to the main loop
                one_delta_pos = index + (self.data_init.index[1]-self.data_init.index[0])
                self.data_init_sub_buy = self.data_init.loc[one_delta_pos:]
                for index_sub_buy, data_sub_buy in self.data_init_sub_buy.iterrows():
                    # initialize flag in every loop
                    sltp_flag = False
                    sl_flag = data_sub_buy.Close <= (data.Close * (1 - sl))  # condition for stop loss in sub loop
                    tp_flag = data_sub_buy.Close > (data.Close * (1 + tp)) # condition for take profit in sub loop
                    sltp_flag = sl_flag or tp_flag # flag once detected sl or tp, to stop the long position loop
                    self.data_init.loc[index_sub_buy,'sl_flag'] = sl_flag
                    self.data_init.loc[index_sub_buy,'tp_flag'] = tp_flag
#                     print(index_sub_buy.strftime("%Y-%m-%d-%H:%M"))
#                     print('data_sub_buy.Close', data_sub_buy.Close)
#                     print('(data.Close * (1 - self.sl))', (data.Close * (1 - self.sl)))
#                     print('data_sub_buy.Close <= (data.Close * (1 - self.sl))', data_sub_buy.Close <= (data.Close * (1 - self.sl)))
#                     print('sl_flag', sl_flag)
#                     print(index_sub_buy.strftime("%Y-%m-%d-%H:%M"))
#                     print('data_sub_buy.Close', data_sub_buy.Close)
#                     print('(data.Close * (1 + self.tp))', (data.Close * (1 + self.tp)))
#                     print('data_sub_buy.Close > (data.Close * (1 + self.tp))', data_sub_buy.Close > (data.Close * (1 + self.tp)))
#                     print('tp_flag', tp_flag)                    
                    # we are still in uptrend and no sltp signal -> keep long and ignore macd signals
                    if (data_sub_buy.sma_conf == 1 and sltp_flag == False):
                        self.data_init.loc[index_sub_buy, 'position'] = 1
                    # if either change in trend or sltp signal appears we record date and break to go to main loop
                    elif (data_sub_buy.sma_conf != 1 or sltp_flag == True):
                        if (sltp_flag == True):
                            self.sltp_dates.append((index_sub_buy,f"sl={sl_flag}",f"tp={tp_flag}"))
                        break
                    # no other conditions expected
                    else:
                        raise Exception('unexpected branch')
            # no need to assess neutral conditions since positions are 0 anyways. 
            # no need to assess sltp since we are not going long in this case. Only long positions aborted by sltp are appended.
            else:
                pass             

        #stablish the trading costs and the number of trades done
        self.data_init['trades'] = 0
        trading_cost = np.log(1 - 0.00075) + np.log(1 - 0.0001)
        trade_exec_cond = self.data_init.position.diff().fillna(0).abs() != 0
        self.data_init.loc[trade_exec_cond, 'trades'] = 1
        total_trades = self.data_init.trades[self.data_init.trades == 1].sum()
        #calculate strategy returns
        self.data_init['macd_log_returns'] = self.data_init.log_returns_hold * self.data_init.position.shift(1)
        self.data_init['macd_log_returns_net'] = self.data_init.macd_log_returns + self.data_init.trades * trading_cost
        self.data_init['macd_log_returns_acum'] = np.exp(self.data_init.macd_log_returns.cumsum())
        self.data_init['macd_log_returns_net_acum'] = np.exp(self.data_init.macd_log_returns_net.cumsum())
     
        #calculating profitable trades with trading costs
        cond_profit_net1 = self.data_init.trades == 1
        cond_profit_net2 = ((self.data_init.Close.div(self.data_init.Close.shift(-1)) - 1) + np.log(1 - 0.00075) + np.log(1 - 0.0001))  > 0
        success_trades_net = self.data_init[cond_profit_net1&cond_profit_net2].trades.sum()
        success_trades_net_pct = round((success_trades_net / total_trades)*100,1)
        
        #calculating failure trades with trading costs
        cond_failure_net1 = self.data_init.trades == 1
        cond_failure_net2 = ((self.data_init.Close.div(self.data_init.Close.shift(-1)) - 1) + np.log(1 - 0.00075) + np.log(1 - 0.0001))  < 0
        failure_trades_net = self.data_init[cond_failure_net1&cond_failure_net2].trades.sum()
        failure_trades_net_pct = round((failure_trades_net / total_trades)*100, 1) 
         
        #multiplier for annualized calculations
        if (self.interval == '1d'):
            m = 1*365
        elif (self.interval == '5m'):
            m = 24*12*365
        elif (self.interval == '15m'):
            m = 24*4*365
        elif (self.interval == '30m'):
            m = 24*2*365
        elif (self.interval == '1h'):
            m = 24*365
            
        #calculating the function outputs
        multiple_hold = np.exp(self.data_init.log_returns_hold.sum())
        ann_log_mean_hold = self.data_init.log_returns_hold.mean() * m
        ann_log_std_hold = self.data_init.log_returns_hold.std() * np.sqrt(m)
        sharpe_ratio_hold = ann_log_mean_hold / ann_log_std_hold
        
        multiple_macd_strategy = np.exp(self.data_init.macd_log_returns.sum())
        ann_log_mean_macd = self.data_init.macd_log_returns.mean() * m
        ann_log_std_macd = self.data_init.macd_log_returns.std() * np.sqrt(m)
        sharpe_ratio_macd = ann_log_mean_macd / ann_log_std_macd
        
        multiple_macd_strategy_net = np.exp(self.data_init.macd_log_returns_net.sum())
        ann_log_mean_macd_net = self.data_init.macd_log_returns_net.mean() * m
        ann_log_std_macd_net = self.data_init.macd_log_returns_net.std() * np.sqrt(m)
        sharpe_ratio_macd_net = ann_log_mean_macd_net / ann_log_std_macd_net
        tuple_return = (multiple_hold, ann_log_mean_hold, ann_log_std_hold, sharpe_ratio_hold, multiple_macd_strategy, ann_log_mean_macd, ann_log_std_macd, sharpe_ratio_macd, multiple_macd_strategy_net, ann_log_mean_macd_net, ann_log_std_macd_net, sharpe_ratio_macd_net, success_trades_net, failure_trades_net, success_trades_net_pct, failure_trades_net_pct, total_trades)

        df_return = pd.DataFrame(data=[list(tuple_return)], columns=['multiple_hold', 'ann_log_mean_hold', 'ann_log_std_hold', 'sharpe_ratio_hold', 'multiple_macd_strategy', 'ann_log_mean_macd', 'ann_log_std_macd', 'sharpe_ratio_macd', 'multiple_macd_strategy_net', 'ann_log_mean_macd_net', 'ann_log_std_macd_net', 'sharpe_ratio_macd_net', 'success_trades_net', 'failure_trades_net', 'success_trades_net_pct', 'failure_trades_net_pct', 'total_trades'])
        print(df_return)
        return tuple_return
    
    def plot_backtest_results(self, start_plot=None, end_plot=None, width_bars=0.1):
        # from IPython.core.display import display, HTML
        # display(HTML("<style>.container { width:100% !important; }</style>"))
        colors=[]

        fig, (close_ax, macd_ax, acum_ax) = plt.subplots(nrows=3, ncols=1, figsize=(30,20), gridspec_kw={'height_ratios': [4,2,4]}, sharex=True)

        close_ax.grid(visible=True, which='major', axis='x', color='grey')
        macd_ax.grid(visible=True, which='major', axis='x', color='grey')
        acum_ax.grid(visible=True, which='major', axis='x', color='grey')
        close_ax.grid(visible=True, which='major', axis='y', color='grey')
        macd_ax.grid(visible=True, which='major', axis='y', color='grey')
        acum_ax.grid(visible=True, which='major', axis='y', color='grey')
        close_ax.grid(visible=True, which='minor', axis='x', color='grey')
        macd_ax.grid(visible=True, which='minor', axis='x', color='grey')
        acum_ax.grid(visible=True, which='minor', axis='x', color='grey')        

        close_ax.tick_params(labelrotation=45)
        macd_ax.tick_params(labelrotation=45)
        acum_ax.tick_params(labelrotation=45)

        close_ax.margins(0)
        macd_ax.margins(0)
        acum_ax.margins(0)
        
        close_ax.set_ylim(auto=True)

#         days = mdates.DayLocator()
#         macd_ax.xaxis.set_minor_locator(days)
#         date_form = mdates.DateFormatter('%Y-%m-%dT%H:%M:%S')
#         close_ax.xaxis.set_major_formatter(date_form)
        
        data_init_ready = self.data_init              
        if ((start_plot != None) and (end_plot !=None)):
            cond_start = self.data_init.index >= start_plot
            cond_end = self.data_init.index <= end_plot
            data_init_ready = self.data_init[cond_start&cond_end]
                
        for index, value in data_init_ready.macd_diff.iteritems():
            if value > 0:
                colors.append('g')
            else:
                colors.append('r')
                
        close_ax.plot(data_init_ready.index, data_init_ready.Close) #plot the data without shifting
        close_ax.plot(data_init_ready.index, data_init_ready.sma_fast_ps)
        close_ax.plot(data_init_ready.index, data_init_ready.sma_slow_ps)
        close_ax.legend(['Close', 'sma_fast_ps', 'sma_slow_ps'])        
        
        #shift one position the inv_sign only for plotting the signal in the day after is found, without shifting the
        #Close prices
        data_init_ready_shift = data_init_ready.copy()
        data_init_ready_shift['position'] = data_init_ready.position.shift(1)        
        neutral_cond = data_init_ready_shift.position == 0
        neutral_trade = data_init_ready_shift.loc[neutral_cond]
        long_cond = data_init_ready_shift.position == 1               
        long_trade = data_init_ready_shift.loc[long_cond]
        close_ax.scatter(neutral_trade.index, neutral_trade.Close.loc[neutral_trade.index], marker='^', color='r', s=100)
        close_ax.scatter(long_trade.index, long_trade.Close.loc[long_trade.index], marker='^', color='g', s=100)
        
        macd_ax.bar(x= data_init_ready.index, height= data_init_ready.macd_diff, width=width_bars, align='center', color=colors, edgecolor='black')

        acum_ax.plot(data_init_ready.index, data_init_ready.multiple_hold_acum)
        acum_ax.plot(data_init_ready.index, data_init_ready.macd_log_returns_acum)  
        acum_ax.legend(['multiple_hold_acum','macd_log_returns_acum'],fontsize=14)
        
    def execute_opt(self, start_opt=None, end_opt=None, ema_slow_opt=None, ema_fast_opt=None, ema_sign_opt=None, sma_slow=None, sma_fast=None, sl=None, tp=None):
        '''
        REMARK: Introduced time must be in Tokyo time (UTC+9) but the calculations will be in UTC
        '''
        
        combinations = list(product(sl, tp))
        results = []
        ini_time = datetime.now()
        
        for comb in combinations:
            try:
                self.prepare_data(start=start_opt, end=end_opt)
                tuple_results = (ema_slow_opt, ema_fast_opt, ema_sign_opt, sma_slow, sma_fast, comb[0], comb[1], *self.execute_backtest(start=self.start, ema_slow=ema_slow_opt, ema_fast=ema_fast_opt, ema_signal=ema_sign_opt, sma_slow=sma_slow, sma_fast=sma_fast, sl=comb[0], tp=comb[1]), start_opt, end_opt)
                results.append(tuple_results)
                print(f"processed {len(results)} out of a total {len(combinations)}")
            except (BinanceAPIException, ConnectionResetError, requests.exceptions.ConnectionError, requests.exceptions.RequestException) as e:
                    print('Something went wrong. Error occured at %s. Wait for 60s.' % (datetime.now().astimezone(timezone.utc)))
                    time.sleep(60)
                    api_key = "Q3Zsx6rO7uy0YntyWjb9CTGxbQAfENBxbPAkeOtksXm2AcLcu1y7IOj1fgPFtutO"
                    api_secret = "LeiOCoJdBuCtfgrt1WfsBweeyC2ZwuogPuDrkXFTioEGoaZYOGkju1GRM3yVqp7v"
                    client = Client(api_key, api_secret)
            except KeyboardInterrupt as e2:
                    print("error type is 'KeyboardInterrupt'. The data calculated so far has been stored in self.opt_results")
                    break
        end_time = datetime.now()
        
        self.opt_minutes_elapsed = ((end_time - ini_time).total_seconds())/60
        
        mg = pd.DataFrame(data=results, columns = ['macd_slow_opt', 'macd_fast_opt', 'macd_signal_opt', 'sma_slow', 'sma_fast', 'sl_opt', 'tp_opt', 'multiple_hold', 'ann_log_mean_hold', 'ann_log_std_hold', 'sharpe_ratio_hold', 'multiple_macd_strategy', 'ann_log_mean_macd', 'ann_log_std_macd', 'sharpe_ratio_macd', 'multiple_macd_strategy_net', 'ann_log_mean_macd_net', 'ann_log_std_macd_net', 'sharpe_ratio_macd_net', 'success_trades_net', 'failure_trades_net', 'success_trades_net_pct', 'failure_trades_net_pct', 'total_trades', 'start_opt', 'end_opt'])
        #Filtering only meaningfull combinations
        cond1 = mg.multiple_macd_strategy != 1 #not enough data to carry out a single crossover
        cond2 = mg.macd_slow_opt > mg.macd_fast_opt
        mg_filt = mg.loc[cond1&cond2].copy()
        self.opt_results = mg_filt.copy()
        self.opt_comb_num = len(combinations)
        self.opt_ini_time_str = ini_time.strftime('%Y-%m-%d-%H:%M')
        
        opt_max = self.opt_results.multiple_macd_strategy[self.opt_results.multiple_macd_strategy == self.opt_results.multiple_macd_strategy.max()]
        opt_max_net = self.opt_results.multiple_macd_strategy_net[self.opt_results.multiple_macd_strategy_net == self.opt_results.multiple_macd_strategy_net.max()]
        self.opt_max = opt_max
        self.opt_max_net = opt_max_net
        
        if os.path.exists(f"macd_sma_{self.opt_ini_time_str}_{self.interval}_{self.symbol}.csv"):
            df_imported = pd.read_csv(f"macd_sma_{self.opt_ini_time_str}_{self.interval}_{self.symbol}.csv")
            df_complete = pd.concat([df_imported, self.opt_results], axis = 0)
            df_complete.to_csv(f"macd_sma_{self.opt_ini_time_str}_{self.interval}_{self.symbol}.csv", mode='w', index=False, header=True)
        else:
            self.opt_results.to_csv(f"macd_sma_{self.opt_ini_time_str}_{self.interval}_{self.symbol}.csv", mode='w', index=False, header=True)    

In [710]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [711]:
test = Macd_long_backtester_sma_sltp(symbol='BTCUSDT', interval='1h')

class version 0.0 is being used


In [712]:
# test.prepare_data(start="2021-06-01-00:00", end="2022-04-01-00:00")

In [716]:
test.execute_opt(start_opt="2022-01-01-00:00", end_opt="2022-04-01-00:00", ema_slow_opt=25, ema_fast_opt=10, ema_sign_opt=25, sma_fast=20, sma_slow=100, sl=np.linspace(0.002, 0.02, 15), tp=np.linspace(0.02, 0.09, 10))

2161
2261
   multiple_hold  ann_log_mean_hold  ann_log_std_hold  sharpe_ratio_hold  \
0       0.995664          -0.017699          0.668302          -0.026483   

   multiple_macd_strategy  ann_log_mean_macd  ann_log_std_macd  \
0                0.929966          -0.295696           0.11351   

   sharpe_ratio_macd  multiple_macd_strategy_net  ann_log_mean_macd_net  \
0          -2.605023                    0.895816               -0.44806   

   ann_log_std_macd_net  sharpe_ratio_macd_net  success_trades_net  \
0              0.117399              -3.816558                  16   

   failure_trades_net  success_trades_net_pct  failure_trades_net_pct  \
0                  28                    36.4                    63.6   

   total_trades  
0            44  
processed 1 out of a total 150
2161
2261
   multiple_hold  ann_log_mean_hold  ann_log_std_hold  sharpe_ratio_hold  \
0       0.995664          -0.017699          0.668302          -0.026483   

   multiple_macd_strategy  ann_log_

2161
2261
   multiple_hold  ann_log_mean_hold  ann_log_std_hold  sharpe_ratio_hold  \
0       0.995664          -0.017699          0.668302          -0.026483   

   multiple_macd_strategy  ann_log_mean_macd  ann_log_std_macd  \
0                0.952223          -0.199374           0.14907   

   sharpe_ratio_macd  multiple_macd_strategy_net  ann_log_mean_macd_net  \
0          -1.337453                    0.917256              -0.351738   

   ann_log_std_macd_net  sharpe_ratio_macd_net  success_trades_net  \
0              0.152605              -2.304896                  18   

   failure_trades_net  success_trades_net_pct  failure_trades_net_pct  \
0                  26                    40.9                    59.1   

   total_trades  
0            44  
processed 12 out of a total 150
2161
2261
   multiple_hold  ann_log_mean_hold  ann_log_std_hold  sharpe_ratio_hold  \
0       0.995664          -0.017699          0.668302          -0.026483   

   multiple_macd_strategy  ann_log

2161
2261
   multiple_hold  ann_log_mean_hold  ann_log_std_hold  sharpe_ratio_hold  \
0       0.995664          -0.017699          0.668302          -0.026483   

   multiple_macd_strategy  ann_log_mean_macd  ann_log_std_macd  \
0                0.956655          -0.180464          0.173515   

   sharpe_ratio_macd  multiple_macd_strategy_net  ann_log_mean_macd_net  \
0          -1.040052                    0.923094              -0.325903   

   ann_log_std_macd_net  sharpe_ratio_macd_net  success_trades_net  \
0              0.176467              -1.846815                  17   

   failure_trades_net  success_trades_net_pct  failure_trades_net_pct  \
0                  25                    40.5                    59.5   

   total_trades  
0            42  
processed 23 out of a total 150
2161
2261
   multiple_hold  ann_log_mean_hold  ann_log_std_hold  sharpe_ratio_hold  \
0       0.995664          -0.017699          0.668302          -0.026483   

   multiple_macd_strategy  ann_log

2161
2261
   multiple_hold  ann_log_mean_hold  ann_log_std_hold  sharpe_ratio_hold  \
0       0.995664          -0.017699          0.668302          -0.026483   

   multiple_macd_strategy  ann_log_mean_macd  ann_log_std_macd  \
0                0.956048          -0.183047          0.179348   

   sharpe_ratio_macd  multiple_macd_strategy_net  ann_log_mean_macd_net  \
0          -1.020622                    0.924078              -0.321559   

   ann_log_std_macd_net  sharpe_ratio_macd_net  success_trades_net  \
0              0.182651               -1.76051                  17   

   failure_trades_net  success_trades_net_pct  failure_trades_net_pct  \
0                  23                    42.5                    57.5   

   total_trades  
0            40  
processed 34 out of a total 150
2161
2261
   multiple_hold  ann_log_mean_hold  ann_log_std_hold  sharpe_ratio_hold  \
0       0.995664          -0.017699          0.668302          -0.026483   

   multiple_macd_strategy  ann_log

2161
2261
   multiple_hold  ann_log_mean_hold  ann_log_std_hold  sharpe_ratio_hold  \
0       0.995664          -0.017699          0.668302          -0.026483   

   multiple_macd_strategy  ann_log_mean_macd  ann_log_std_macd  \
0                0.989836          -0.041603          0.188113   

   sharpe_ratio_macd  multiple_macd_strategy_net  ann_log_mean_macd_net  \
0           -0.22116                    0.959996              -0.166264   

   ann_log_std_macd_net  sharpe_ratio_macd_net  success_trades_net  \
0              0.190594              -0.872347                  15   

   failure_trades_net  success_trades_net_pct  failure_trades_net_pct  \
0                  21                    41.7                    58.3   

   total_trades  
0            36  
processed 45 out of a total 150
2161
2261
   multiple_hold  ann_log_mean_hold  ann_log_std_hold  sharpe_ratio_hold  \
0       0.995664          -0.017699          0.668302          -0.026483   

   multiple_macd_strategy  ann_log

...

2261
   multiple_hold  ann_log_mean_hold  ann_log_std_hold  sharpe_ratio_hold  \
0       0.995664          -0.017699          0.668302          -0.026483   

   multiple_macd_strategy  ann_log_mean_macd  ann_log_std_macd  \
0                0.920592          -0.336953          0.265855   

   sharpe_ratio_macd  multiple_macd_strategy_net  ann_log_mean_macd_net  \
0          -1.267433                    0.899698              -0.430449   

   ann_log_std_macd_net  sharpe_ratio_macd_net  success_trades_net  \
0               0.26711              -1.611509                  13   

   failure_trades_net  success_trades_net_pct  failure_trades_net_pct  \
0                  14                    48.1                    51.9   

   total_trades  
0            27  
processed 100 out of a total 150
2161
2261
   multiple_hold  ann_log_mean_hold  ann_log_std_hold  sharpe_ratio_hold  \
0       0.995664          -0.017699          0.668302          -0.026483   

   multiple_macd_strategy  ann_log_mea

2261
   multiple_hold  ann_log_mean_hold  ann_log_std_hold  sharpe_ratio_hold  \
0       0.995664          -0.017699          0.668302          -0.026483   

   multiple_macd_strategy  ann_log_mean_macd  ann_log_std_macd  \
0                0.995367          -0.018912          0.190508   

   sharpe_ratio_macd  multiple_macd_strategy_net  ann_log_mean_macd_net  \
0          -0.099273                     0.96536              -0.143574   

   ann_log_std_macd_net  sharpe_ratio_macd_net  success_trades_net  \
0              0.190187              -0.754907                  14   

   failure_trades_net  success_trades_net_pct  failure_trades_net_pct  \
0                  21                    38.9                    58.3   

   total_trades  
0            36  
processed 111 out of a total 150
2161
2261
   multiple_hold  ann_log_mean_hold  ann_log_std_hold  sharpe_ratio_hold  \
0       0.995664          -0.017699          0.668302          -0.026483   

   multiple_macd_strategy  ann_log_mea

2161
2261
   multiple_hold  ann_log_mean_hold  ann_log_std_hold  sharpe_ratio_hold  \
0       0.995664          -0.017699          0.668302          -0.026483   

   multiple_macd_strategy  ann_log_mean_macd  ann_log_std_macd  \
0                0.945675          -0.227475          0.221904   

   sharpe_ratio_macd  multiple_macd_strategy_net  ann_log_mean_macd_net  \
0          -1.025105                    0.918727              -0.345211   

   ann_log_std_macd_net  sharpe_ratio_macd_net  success_trades_net  \
0              0.223161              -1.546912                  14   

   failure_trades_net  success_trades_net_pct  failure_trades_net_pct  \
0                  19                    41.2                    55.9   

   total_trades  
0            34  
processed 122 out of a total 150
2161
2261
   multiple_hold  ann_log_mean_hold  ann_log_std_hold  sharpe_ratio_hold  \
0       0.995664          -0.017699          0.668302          -0.026483   

   multiple_macd_strategy  ann_lo

2261
   multiple_hold  ann_log_mean_hold  ann_log_std_hold  sharpe_ratio_hold  \
0       0.995664          -0.017699          0.668302          -0.026483   

   multiple_macd_strategy  ann_log_mean_macd  ann_log_std_macd  \
0                0.838774          -0.716007          0.249756   

   sharpe_ratio_macd  multiple_macd_strategy_net  ann_log_mean_macd_net  \
0          -2.866828                    0.815566               -0.83028   

   ann_log_std_macd_net  sharpe_ratio_macd_net  success_trades_net  \
0              0.251176              -3.305571                  12   

   failure_trades_net  success_trades_net_pct  failure_trades_net_pct  \
0                  21                    36.4                    63.6   

   total_trades  
0            33  
processed 133 out of a total 150
2161
2261
   multiple_hold  ann_log_mean_hold  ann_log_std_hold  sharpe_ratio_hold  \
0       0.995664          -0.017699          0.668302          -0.026483   

   multiple_macd_strategy  ann_log_mea

2161
2261
   multiple_hold  ann_log_mean_hold  ann_log_std_hold  sharpe_ratio_hold  \
0       0.995664          -0.017699          0.668302          -0.026483   

   multiple_macd_strategy  ann_log_mean_macd  ann_log_std_macd  \
0                0.846203          -0.680097          0.250418   

   sharpe_ratio_macd  multiple_macd_strategy_net  ann_log_mean_macd_net  \
0          -2.715852                    0.824189              -0.787444   

   ann_log_std_macd_net  sharpe_ratio_macd_net  success_trades_net  \
0              0.252039              -3.124296                  12   

   failure_trades_net  success_trades_net_pct  failure_trades_net_pct  \
0                  19                    38.7                    61.3   

   total_trades  
0            31  
processed 144 out of a total 150
2161
2261
   multiple_hold  ann_log_mean_hold  ann_log_std_hold  sharpe_ratio_hold  \
0       0.995664          -0.017699          0.668302          -0.026483   

   multiple_macd_strategy  ann_lo

In [717]:
test.opt_max

26    1.068273
Name: multiple_macd_strategy, dtype: float64

In [720]:
test.opt_results.iloc[26]

macd_slow_opt                               25
macd_fast_opt                               10
macd_signal_opt                             25
sma_slow                                   100
sma_fast                                    20
sl_opt                                0.004571
tp_opt                                0.066667
multiple_hold                         0.995664
ann_log_mean_hold                    -0.017699
ann_log_std_hold                      0.668302
sharpe_ratio_hold                    -0.026483
multiple_macd_strategy                1.068273
ann_log_mean_macd                     0.268962
ann_log_std_macd                      0.213755
sharpe_ratio_macd                     1.258275
multiple_macd_strategy_net            1.039598
ann_log_mean_macd_net                 0.158152
ann_log_std_macd_net                  0.215519
sharpe_ratio_macd_net                  0.73382
success_trades_net                          15
failure_trades_net                          17
success_trade

In [691]:
df = test.data_init.describe()
df

,Open,High,Low,Close,Volume,macd_diff,macd_macd,macd_signal,sma_fast_ps,sma_slow_ps,log_returns_hold,multiple_hold_acum,position,inv_sign,sma_conf,trades,macd_log_returns,macd_log_returns_net,macd_log_returns_acum,macd_log_returns_net_acum
count,2152.000000,2152.000000,2152.000000,2152.000000,2152.000000,2152.000000,2152.000000,2152.000000,2152.000000,2152.000000,2151.000000,2151.000000,2152.000000,2152.000000,2152.000000,2152.000000,2151.000000,2151.000000,2151.000000,2151.000000
mean,41220.788230,41428.381004,41009.707045,41220.901492,1865.830612,0.173869,-0.373401,-0.547269,41220.408964,41223.022860,-0.000002,0.883450,0.310409,0.000000,0.035316,0.008829,-0.000035,-0.000043,0.917818,0.908962
std,3095.692181,3073.272603,3116.606529,3095.885807,1522.335908,216.123872,379.895985,306.167303,3049.426876,2850.785171,0.007140,0.066323,0.462769,0.240099,0.999608,0.093569,0.003273,0.003282,0.043894,0.047328
min,33114.890000,33776.020000,32917.170000,33114.900000,301.689100,-830.288936,-1202.660539,-1008.472518,34738.157500,35650.675500,-0.044597,0.709765,0.000000,-1.000000,-1.000000,0.000000,-0.034206,-0.035057,0.837299,0.823881
25%,38741.095000,38947.365000,38532.787500,38741.285000,937.290525,-119.007917,-186.277906,-168.705893,38728.327625,38952.756275,-0.003174,0.830357,0.000000,0.000000,-1.000000,0.000000,0.000000,-0.000000,0.864739,0.853054
50%,41640.420000,41811.025000,41469.995000,41641.425000,1387.493290,2.452688,0.276442,2.345749,41634.532000,41574.419950,0.000071,0.892505,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.914383,0.903562
75%,43252.772500,43475.020000,43048.920000,43252.752500,2241.100780,115.961594,188.904900,179.120237,43179.518250,42967.100525,0.003327,0.927011,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.960804,0.955915
max,47970.980000,48189.840000,47811.400000,47970.990000,13411.254931,961.048356,1477.853554,994.590959,47607.535000,47777.006000,0.046877,1.028182,1.000000,1.000000,1.000000,1.000000,0.029183,0.029183,1.012951,1.010370


__In order to calculate the sltp intervals for the optimization, the difference of the 50% of High and Low prices is measured, to have an idea of how much can change the price and hence the sltp interval__

In [693]:
high_avg = df.High.loc['50%']
high_avg

41811.025

In [694]:
low_avg = df.Low.loc['50%']
low_avg

41469.994999999995

In [701]:
1-(low_avg/high_avg)

0.008156461124787184

In [669]:
df_mod = df.Close.to_frame()

In [670]:
df_mod

,Close
count,7282.000000
mean,45322.056445
std,8845.609558
min,29238.990000
25%,38682.487500
50%,43942.830000
75%,49244.515000
max,68633.690000


In [675]:
df.Close.loc['max'] / df.Close.loc['min']

2.347334500952324

In [679]:
df.Close.loc['std'] / df.Close.loc['mean']

0.1951722903146441